# 20200821 サブゼミ　効果検証入門 p49~59

効果検証入門の本文中で行われている分析を再現しようとしたが、推定値が合わなかったので、修正。
主な原因は、**データに人工的なバイアスを発生させていなかった**こと。第１章で記述されていたバイアスの作り方を第２章で使うデータセットにも適用しないと、本と同じ推定結果が得られない。

___

まずは必要なライブラリ・モジュール群をインポート。

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import sys

### データの生成

データのインポート、女性向けメールを配信したレコードの削除、バイアスの発生

In [12]:
# セレクションバイアスのあるデータの作成

###データのインポート
mail_df = pd.read_csv("../datas/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20 (1).csv")
### 女性向けメールが配信されたデータを削除したデータを作成
male_df = mail_df[mail_df.segment != 'Womens E-Mail'].copy()  # 女性向けメールが配信されたデータを削除
male_df["treatment"] = np.where(male_df.segment =="Mens E-Mail",1,0)#介入変数（処置変数）Z

## バイアスのあるデータの作成
sample_rules = (male_df.history > 300) | (male_df.recency < 6) | (male_df.channel == 'Multichannel')
biased_df = pd.concat([
    male_df[(sample_rules) & (male_df.treatment == 0)].sample(frac=0.5, random_state=1),
    male_df[(sample_rules) & (male_df.treatment == 1)],
    male_df[(~sample_rules) & (male_df.treatment == 0)],
    male_df[(~sample_rules) & (male_df.treatment == 1)].sample(frac=0.5, random_state=1)
], axis=0, ignore_index=True)

`segment`カラムの値が`Womens E-mail`以外のデータレコードのみ残すクエリ、`np.where`で介入変数のカラムを追加するクエリをそれぞれ記述。

以下、RCTデータ（バイアスなし）：`male_df`
バイアスありデータ：`biased_df`

In [5]:
#カラム内のユニークな要素を確認。
print(np.unique(biased_df["segment"]))

['Mens E-Mail' 'No E-Mail']


Numpyの`unique`関数を用いて、`segment`カラムに含まれるユニークな要素をリストアップする。この操作でユニークなカテゴリの種類を把握・確認する。今回は予想通り、２通りのカテゴリしかないことが確認できたが、**汚いデータだと表記の揺れやスペースの全角と半角の違いなどで違う値として含まれている**可能性がある。その場合は、表記を統一する必要がある。この操作は実務上では重要な確認事項。

___

## RCTデータでの回帰分析

今回使うデータセットは、RCT（ABテスト）を行ったものであるので、セレクション・バイアスは生じていないと考えて良い。つまり、目的変数を介入変数のみで回帰して得られる推定値は、純粋な処置効果の推定値として解釈できる。

In [6]:
#RCTデータでの単回帰分析
Y_1 = male_df[['spend']]
X_1 = male_df[['treatment']]
X_1 = sm.add_constant(X_1)
model = sm.OLS(Y_1,X_1)
results = model.fit()
results.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.6528,0.103,6.356,0.000,0.451,0.854
treatment,0.7698,0.145,5.300,0.000,0.485,1.055


`statsmodels.api`を用いて回帰分析を行った。Rの操作と異なるのは、

- **定数項**を含む回帰モデルであることを明示的に示さなければならない。
- カテゴリ変数を**自動的にダミー変数化することはできない**（Rは自動的にやってくれる）

treatmentの係数は約0.77となり、本の結果とほぼ一致している。これを真の値と考えよう。

___

## バイアスを含むデータでの回帰分析

今度は人工的にバイアスを発生させたデータ(`biased_df`)で回帰分析してみる。

### 単回帰分析

In [15]:
#セレクションバイアスのあるデータでの単回帰分析
Y_2 = biased_df[['spend']]
X_2 = biased_df[['treatment']]
X_2 = sm.add_constant(X_2)
model = sm.OLS(Y_2,X_2)
results = model.fit()
results.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.5580,0.129,4.328,0.000,0.305,0.811
treatment,0.9837,0.176,5.596,0.000,0.639,1.328


treatmentの推定値は0.9837である。0.77と比べて過剰推定した形となっている。ここに、共変量を加えて重回帰分析を試みる。
___


### 重回帰分析

In [16]:
#セレクション・バイアスのあるデータでの重回帰分析
Y_3 = biased_df[['spend']]
X_3 = pd.get_dummies(biased_df[['treatment','recency','channel','history']],columns=['channel'],drop_first=True)
X_3 = sm.add_constant(X_3)
results = sm.OLS(Y_3,X_3).fit()
results.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.4761,0.386,1.233,0.218,-0.281,1.233
treatment,0.8617,0.181,4.750,0.000,0.506,1.217
recency,-0.0361,0.026,-1.372,0.170,-0.088,0.015
history,0.0010,0.000,2.655,0.008,0.000,0.002
channel_Phone,-0.0079,0.310,-0.025,0.980,-0.616,0.600
channel_Web,0.2540,0.310,0.820,0.412,-0.353,0.861


treatmentの推定値は、0.8617である。単回帰分析の結果よりも0.77に近づく結果となった。共変量による交絡調整がうまく行っているようである。

また、pandasの`get_dummies`関数により、channnelカラムのカテゴリ3つに対し、2つのダミー変数を自動的に作成している。Rでの`lm`関数の振る舞いをこれで再現している形。解析はやっぱりRの方が圧倒的に便利。
___
___


## 脱落変数バイアス(OVB)
本来考慮すべき共変量がモデルに含まれていないことにより、興味のある変数に生じてしまうバイアスを**脱落変数バイアス(Omitted Variable Bias)**と呼ぶ（**欠落変数バイアス**と訳されることもあり、個人的にはこっちをよく使う）。この節では、脱落変数バイアスの存在を確かめるためのシミュレーションを行う。

※添字$i$は省略。

モデルA
$$
Spend = \alpha_{0} + \alpha_{1}treatment+ \alpha_{2}recency+ \alpha_{3}channel+e
$$

モデルB
$$
Spend = \beta_{0} + \beta_{1}treatment+ \beta_{2}recency+ \beta_{3}channel+\beta_{4}history+u
$$

の2つを考える。Aは共変量historyが欠落している。この場合、それぞれのモデルのtreatmentの係数$\alpha_{1}$と$\beta_{1}$の間にはどのような関係性があるだろうか。結論としては、

$$
\alpha_{1} =\beta_{1}+\gamma_{1}\beta_{4}
$$

と書くことができる。直感的には、**(欠落しているモデルの係数)=（欠落していないモデルの係数）＋（バイアス）**
と解釈できる。この式における$\gamma_{1}$とは何だろうか？これは、

$$
history = \gamma_{0} + \gamma_{1}treatment+ \gamma_{2}recency+ \gamma_{3}channel+\epsilon
$$

という回帰モデル(モデルCとする)におけるtreatmentの係数である。つまり$\gamma_{1}\beta_{4}$とは、**「欠落変数と介入変数との相関」と「欠落変数と目的変数との相関」との積**である。**欠落変数を一般に$X_{omit}$とすると、$X_{omit}$と$Z$との相関の大きさと$X_{omit}$と$Y$の相関の大きさを掛け算した分だけ過剰に推定してしまう**ことになる。

一方**RCTデータの場合、介入は共変量に依存せずランダムに割り当てられる。したがって$\gamma_{1}=0$となり、OVBは発生しない**。

___

### historyなしの回帰分析

In [19]:
#historyなしの回帰分析(モデルA)
Y_4 = biased_df[['spend']]
X_4 = pd.get_dummies(biased_df[['treatment','recency','channel']],columns=['channel'],drop_first=True)
X_4 = sm.add_constant(X_4)
results = sm.OLS(Y_4,X_4).fit()
results.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,1.0644,0.316,3.365,0.001,0.444,1.684
treatment,0.8905,0.181,4.917,0.000,0.536,1.245
recency,-0.0506,0.026,-1.965,0.049,-0.101,-0.000
channel_Phone,-0.3197,0.287,-1.114,0.265,-0.882,0.243
channel_Web,-0.0576,0.287,-0.201,0.841,-0.619,0.504


historyが欠落したモデルの推定結果。

___

### historyありの回帰分析

In [22]:
#historyありの回帰分析(モデルB)
Y_5 = biased_df[['spend']]
X_5 = pd.get_dummies(biased_df[['treatment','recency','channel','history']],columns=['channel'],drop_first=True)
X_5 = sm.add_constant(X_5)
results = sm.OLS(Y_5,X_5).fit()
results.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.4761,0.386,1.233,0.218,-0.281,1.233
treatment,0.8617,0.181,4.750,0.000,0.506,1.217
recency,-0.0361,0.026,-1.372,0.170,-0.088,0.015
history,0.0010,0.000,2.655,0.008,0.000,0.002
channel_Phone,-0.0079,0.310,-0.025,0.980,-0.616,0.600
channel_Web,0.2540,0.310,0.820,0.412,-0.353,0.861


In [25]:
beta_1 = results.params['treatment']
beta_4 = results.params['history']#脱落変数のパラメータ

historyが欠落していないモデルの推定結果。
___

### historyを介入変数で回帰

In [26]:
#脱落した変数と介入変数の回帰分析(モデルC)
Y_6 = biased_df[['history']]
X_6 = pd.get_dummies(biased_df[['treatment', 'recency', 'channel']], columns=['channel'], drop_first=True)
X_6 = sm.add_constant(X_6)
results = sm.OLS(Y_6, X_6).fit()
modelC_coef = results.summary().tables[1]
gamma_1 = results.params['treatment']

この結果から、$\gamma_{1}$を得る。

これで必要なパラメータは全て得られたので、OVBの計算結果が合うかを計算する。

$$
\alpha_{1} =\beta_{1}+\gamma_{1}\beta_{4}
$$

を変形して

$$
\alpha_{1} -\beta_{1}=\gamma_{1}\beta_{4}
$$

として、右辺と左辺の結果が一致するかを確かめる。つまり、**(モデル同士の係数の差)＝(脱落変数バイアス)**という関係が成り立つかを確かめる。


In [27]:
print(gamma_1*beta_4)
print(alpha_1-beta_1)

0.028816423676830263
0.028816423676823355


計算結果が一致した。数値も本とほぼ同じ値になった。